In [8]:
import requests
import re

url = 'https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieDtl.do'

# 포스트 요청의 본문에 해당하는 데이터
payload = {
    'code': '20247450',
    'sType': '', # 검색 타입을 지정하는 필드 
    'titleYN': 'Y', # 영화 제목 포함 여부 
    'etcParam': '',
    'isOuterReq': 'false',
    'CSRFToken': 'j62iniJVHjKeqJIqvWlnt9lZI10iQDtDd_uVBMUH72M', # csrf 공격을 방지하기 위한 토큰 
}

# 추가적인 메타데이터를 포함
# 클라이언트의 정보를 서버에 전달, 서버가 요청을 올바르게 처리하도록 한다 
# 클라이언트의 유효성 검증을 위해 사용 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Referer': 'https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do', # 요청이 발생한 출처를 서버에 알린다 
    'Cookie': 'JSESSIONID=6TbBnpld1ngM47CxslWzTzJgC4QmVy6cRVgMYLwvycLLBjLrWJJh!1013290353!-1702303710',  # 세션 쿠키
    'X-Requested-With': 'XMLHttpRequest', # ajax 요청임을 명시 
}

response = requests.post(url, headers=headers, data=payload)
# 빈 줄 제거 (줄 사이 공백 제거)
cleaned_text = "\n".join(line for line in response.text.splitlines() if line.strip())
print(cleaned_text)

<!--[if IE]>
<script src="/kobis/web/comm/commjs/html5shiv.js"></script>
<![endif]-->
<!-- 기본 css -->
<link rel="stylesheet" type="text/css" href="/kobis/web/comm/commcss/default.css">
<link rel="stylesheet" type="text/css" href="/kobis/web/comm/commcss/jquery-ui-1.12.1.min.css" />
<link rel="stylesheet" type="text/css" href="/kobis/web/comm/commcss/common.css" />
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery-1.12.4.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery.dotdotdot.min.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery.bxslider.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/comm_script.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery-ui-1.13.2.min.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/jquery.ui.datepicker-ko.js"></script>
<script type="text/javascript" src="/kobis/web/comm/commjs/kobis.js"></script>
<link re